In [1]:
from netort.data_manager import DataSession
import numpy as np
import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.DEBUG)

def get_uploader(data_session, column_mapping, hostname, source):
    """

    :type data_session: DataSession
    """
    # router = {col_name: data_session.new_tank_metric('metrics', metric_name,
    #                                                  hostname, group, source)
    #           for col_name, (metric_name, group) in column_mapping}
    _router = {}
    def get_router(tags):
        if set(tags) - set(_router.keys()):
            [_router.setdefault(tag,
                               {col_name:
                                    data_session.new_tank_metric('metrics',
                                                                 name,
                                                                 hostname,
                                                                 group,
                                                                 source,
                                                                 ammo_tag=tag)
                                for col_name, (name, group) in column_mapping.items()})
             for tag in tags]
        return _router
    
    def upload_df(df):
        router = get_router(df.tag.unique().tolist())
        for tag, df_tag in df.groupby('tag'):
            for col_name, metric in router[tag].items():
                df_tag['value'] = df_tag[col_name]
                metric.put(df_tag)
                
    return upload_df

In [2]:
from netort.data_manager import DataSession
clients = [{'type': 'luna', 'api_address': 'https://volta-back.yandex-team.ru/'}]
data_session = DataSession({'clients': clients})
col_map = {name: (name, 'fractions') for name in ['connect_time', 'send_time',
           'latency', 'receive_time',
           'interval_event']}
uploader = get_uploader(data_session, col_map, 'localhost', 'tank')

INFO:Created new local data session: job_ef674818-acda-4ec6-9fb8-cf34d0140121


DEBUG:Luna client local id: client_c5c63521-a72d-4600-97b7-a8e6a9e98964


DEBUG:No subscribers/callbacks for metrics yet... skipped metrics


DEBUG:Metrics for subscriber subscriber_5311a907-da0a-48cb-b75c-8057585ab15f not found


DEBUG:DataSession clients: [<netort.data_manager.clients.luna.LunaClient object at 0x10a075b10>]


DEBUG:DataSession subscribers:                                                                                           callback  \
subscriber_5311a907-da0a-48cb-b75c-8057585ab15f  <bound method LunaClient.put of <netort.data_m...   

                                                    type  
subscriber_5311a907-da0a-48cb-b75c-8057585ab15f  __ANY__  


In [3]:
from yandextank.plugins.Phantom.reader import string_to_df_microsec
with open('/Users/fomars/dev/netort/tmp/phout_tagged.log') as f:
    chunk = f.read()
df = string_to_df_microsec(chunk)

DEBUG:Luna client processing took 1003.53 ms


DEBUG:No subscribers/callbacks for metrics yet... skipped metrics


Traceback (most recent call last):
  File "/Users/fomars/dev/netort/netort/netort/resource.py", line 16, in <module>
    import boto
ImportError: No module named boto


In [4]:
uploader(df)

DEBUG:Found subscribers for this metric, subscribing...:                                                                                           callback  \
subscriber_5311a907-da0a-48cb-b75c-8057585ab15f  <bound method LunaClient.put of <netort.data_m...   

                                                    type  
subscriber_5311a907-da0a-48cb-b75c-8057585ab15f  __ANY__  


DEBUG:Found subscribers for this metric, subscribing...:                                                                                           callback  \
subscriber_5311a907-da0a-48cb-b75c-8057585ab15f  <bound method LunaClient.put of <netort.data_m...   

                                                    type  
subscriber_5311a907-da0a-48cb-b75c-8057585ab15f  __ANY__  


DEBUG:Found subscribers for this metric, subscribing...:                                                                                           callback  \
subscriber_5311a907-da0a-48cb-b75c-8057585ab15f  <bound method LunaClient.put of <netort.data_m...   

                                                    type  
subscriber_5311a907-da0a-48cb-b75c-8057585ab15f  __ANY__  


DEBUG:Found subscribers for this metric, subscribing...:                                                                                           callback  \
subscriber_5311a907-da0a-48cb-b75c-8057585ab15f  <bound method LunaClient.put of <netort.data_m...   

                                                    type  
subscriber_5311a907-da0a-48cb-b75c-8057585ab15f  __ANY__  


DEBUG:Found subscribers for this metric, subscribing...:                                                                                           callback  \
subscriber_5311a907-da0a-48cb-b75c-8057585ab15f  <bound method LunaClient.put of <netort.data_m...   

                                                    type  
subscriber_5311a907-da0a-48cb-b75c-8057585ab15f  __ANY__  


...

DEBUG:Prepared create_job request:
-----------QUERY START-----------
POST https://volta-back.yandex-team.ru//create_job/
User-Agent: Uploader/0.2.8, 
Content-Length: 27
Content-Type: application/x-www-form-urlencoded

test_start=1547034076999414
-----------QUERY END-----------


DEBUG:Luna client processing took 1016.80 ms


DEBUG:Starting new HTTPS connection (1): volta-back.yandex-team.ru


Exception in thread Thread-4:
Traceback (most recent call last):
  File "/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/threading.py", line 810, in __bootstrap_inner
    self.run()
  File "/Users/fomars/dev/netort/netort/netort/data_manager/router.py", line 29, in run
    self.__route()
  File "/Users/fomars/dev/netort/netort/netort/data_manager/router.py", line 44, in __route
    self.routing_buffer[type_] = pd.concat([self.routing_buffer[type_], df], sort=False)
TypeError: concat() got an unexpected keyword argument 'sort'



DEBUG:Luna client processing took 1185.31 ms
